### **A. Import Required Libraries**

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, precision_score, recall_score
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

### **B. Upload / access the dataset and Preprocessing**

In [2]:
FILE_PATH = 'creditcard.csv'

# Load the dataset
df = pd.read_csv(FILE_PATH)

# 1. Separate features (X) and target (y)
X = df.drop(['Time', 'Class'], axis=1)
y = df['Class']

# 2. Scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
INPUT_DIM = X_scaled.shape[1] # Number of features = 29 (V1 to V28 + Amount)

# 3. Isolate NORMAL (non-fraudulent) transactions for training and validation
X_normal = X_scaled[y == 0]
X_train_normal, X_val_normal = train_test_split(
    X_normal,
    test_size=0.2,
    random_state=42
)

print(f"Data ready. Input dimension: {INPUT_DIM} features.")
print(f"Training Autoencoder on {X_train_normal.shape[0]} normal transactions.")

Data ready. Input dimension: 29 features.
Training Autoencoder on 227452 normal transactions.


### **C. Encoder converts it into latent representation**

In [3]:
LATENT_DIM = 14     # Bottleneck size (29 / 2)
INTERMEDIATE_DIM = 24

# Define the ENCODER Network
# Input Layer
input_layer = Input(shape=(INPUT_DIM,), name='Input_Layer')

# Compressed Layer 1
encoded = Dense(INTERMEDIATE_DIM, activation='relu', name='Encoder_L1')(input_layer)

# Latent Representation (Bottleneck)
latent_representation = Dense(LATENT_DIM, activation='relu', name='Latent_Representation')(encoded)

print("Encoder defined.")

Encoder defined.


### **D. Decoder Converts Back to Original Input**

In [4]:
# Define the DECODER Network
# Decompressed Layer 1 (Symmetrical to Encoder_L1)
decoded = Dense(INTERMEDIATE_DIM, activation='relu', name='Decoder_L1')(latent_representation)

# Output Layer (Must match the Input Dimension)
output_layer = Dense(INPUT_DIM, activation='linear', name='Output_Reconstruction')(decoded)

# ---------------------------------------

# Create the Full Autoencoder Model
autoencoder = Model(inputs=input_layer, outputs=output_layer, name='Anomaly_Autoencoder')

print("Decoder and Full Autoencoder Model defined.")

Decoder and Full Autoencoder Model defined.


### **E. Compile the Model**

In [5]:
autoencoder.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mse', # Mean Squared Error is the metric for reconstruction quality
    metrics=['accuracy']
)

# Display the model architecture
autoencoder.summary()

Model: "Anomaly_Autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Input_Layer (InputLayer)        │ (None, 29)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Encoder_L1 (Dense)              │ (None, 24)             │           720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Latent_Representation (Dense)   │ (None, 14)             │           350 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Decoder_L1 (Dense)              │ (None, 24)             │           360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output_Reconstruction (Dense)   │ (None, 29)             │           725 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,155 (8.42 KB)

 Trainable params: 2,155 (8.42 KB)

 Non-trainable params: 0 (0.00 B)

### **F. Train the Model**

In [6]:
# Note that the input and output are identical (X_train_normal, X_train_normal), as the goal is self-reconstruction.

print("\nStarting Autoencoder model training...")
EPOCHS = 20
BATCH_SIZE = 128

H_auto = autoencoder.fit(
    X_train_normal, X_train_normal,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val_normal, X_val_normal),
    shuffle=True,
    verbose=1
)
print("Autoencoder model training complete.")


Starting Autoencoder model training...
Epoch 1/20
1777/1777 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.3312 - loss: 0.5495 - val_accuracy: 0.4220 - val_loss: 0.3760
Epoch 2/20
1777/1777 ━━━━━━━━━━━━━━━━━━━━ 2s 959us/step - accuracy: 0.4550 - loss: 0.3180 - val_accuracy: 0.4776 - val_loss: 0.2862
Epoch 3/20
1777/1777 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.4942 - loss: 0.2734 - val_accuracy: 0.5014 - val_loss: 0.2618
Epoch 4/20
1777/1777 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5198 - loss: 0.2496 - val_accuracy: 0.5421 - val_loss: 0.2314
Epoch 5/20
1777/1777 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5522 - loss: 0.2267 - val_accuracy: 0.5503 - val_loss: 0.2236
Epoch 6/20
1777/1777 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5707 - loss: 0.2106 - val_accuracy: 0.5853 - val_loss: 0.2012
Epoch 7/20
1777/1777 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5870 - loss: 0.1951 - val_accuracy: 0.5849 - val_loss: 0.1955
Epoch 8/20
1777/1777 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms

### **G. Calculate Reconstruction Error (Anomaly Score)**

Our **Autoencoder Model Predicts the features** (not target) given the features itself (It tries to reconstruct the input values as it is).

**Error rates are low** (close to 0) when model reconstructs non-fraudulent transaction's features as it is familiar with these patterns (we train the model only on non-fraudulent data).

**Fraud transactions have a larger error rate** as the model is not familiar with these patterns. (they are like 'out of syllabus' questions).

In [7]:
# Get reconstructions for the entire scaled dataset (normal and fraud)
reconstructions = autoencoder.predict(X_scaled)

# Calculate the Mean Squared Error (MSE) for each transaction
mse = np.mean(np.square(X_scaled - reconstructions), axis=1)

# Store results in a DataFrame for easy analysis
error_df = pd.DataFrame({
    'Reconstruction_Error': mse,
    'True_Class': y
})

8901/8901 ━━━━━━━━━━━━━━━━━━━━ 4s 424us/step


In [8]:
fraud_errors = error_df[error_df['True_Class'] == 1]
normal_errors = error_df[error_df['True_Class'] == 0]

print(fraud_errors.tail())
print("\n")
print(normal_errors.tail())

        Reconstruction_Error  True_Class
279863              5.796587           1
280143              2.927265           1
280149              2.940434           1
281144              5.496614           1
281674              0.042834           1


        Reconstruction_Error  True_Class
284802              0.230511           0
284803              0.036120           0
284804              0.006130           0
284805              0.124024           0
284806              0.061479           0


### **H. Evaluation**

We find a **THRESHOLD** value for all the errors to be compared with.

This is the **value which is greater than 95% of the error values** of all **non-fraudulent** transactions.

This also means that all other transactions with **error > THRESHOLD** will be considered **FRAUD** (including 5% normal transactions)

In [9]:
# Extract the normal (non-fraudulent) reconstruction errors
normal_error = error_df[error_df['True_Class'] == 0].Reconstruction_Error

# 1. Set Anomaly Threshold
# Use the 95th percentile of the reconstruction error from NORMAL transactions
THRESHOLD = np.percentile(normal_error, 95)
print(f"\nCalculated Anomaly Threshold: {THRESHOLD:.6f}")

# 2. Predict anomalies for the entire dataset
# The prediction is TRUE (1 or Fraud) if the error is above the threshold
predicted_anomalies = error_df['Reconstruction_Error'] > THRESHOLD


Calculated Anomaly Threshold: 0.459735


In [10]:
print("\nConfusion Matrix")
print(confusion_matrix(error_df['True_Class'], predicted_anomalies))


Confusion Matrix
[[270099  14216]
 [    73    419]]


In [11]:
# Calculate and print Precision for the minority class (pos_label=1)
precision = precision_score(error_df['True_Class'], predicted_anomalies, pos_label=1)
print(f"Precision: {100*precision:.2f}%")

# Calculate and print Recall for the minority class (pos_label=1)
recall = recall_score(error_df['True_Class'], predicted_anomalies, pos_label=1)
print(f"Recall: {100*recall:.2f}%")

Precision: 2.86%
Recall: 85.16%


Here, the **main evaluation metric is Recall** and not Precision.

High Recall indicates that higher number of Fraud transactions have been correctly flagged, which is the main goal.